In [22]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
import numpy as np
import json
from  tensorflow import keras
from tensorflow.keras import layers
import os
import tqdm as tqdm

In [23]:
shape =  np.stack(np.load('./processed_quickdraw/octopus.npy', allow_pickle = True)).shape

In [24]:

total_data = np.empty((0, shape[1], shape[2]))
y = np.array([])
#load  all datasets
input_folder = './tempsmall'
files = os.listdir(input_folder)
for idx, filename in enumerate(tqdm.tqdm(files, desc='Processing files', unit='file')):
    if filename.endswith(".npy"):
        data =  np.stack(np.load(os.path.join(input_folder, filename), allow_pickle = True))
        total_data = np.vstack((total_data, data))
        y = np.concatenate((y, np.ones(data.shape[0])*idx))
X = total_data
y = keras.utils.to_categorical(y)

Processing files: 100%|██████████| 4/4 [00:04<00:00,  1.03s/file]


In [25]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [26]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.__version__)
print(tf.test.is_built_with_gpu_support())

Num GPUs Available:  1
2.10.0
True


In [27]:
# X = np.load('./processed_quickdraw/total_data.npy', allow_pickle= True)

In [28]:
X = X[:,:100,:]

In [29]:
X.shape

(4000, 100, 3)

In [30]:
X

array([[[ 2.66666710e-02, -9.87653434e-03,  0.00000000e+00],
        [ 1.86666548e-02,  0.00000000e+00,  0.00000000e+00],
        [ 2.93333530e-02, -2.46913731e-03,  0.00000000e+00],
        ...,
        [-1.33333206e-02, -1.48148239e-02,  0.00000000e+00],
        [-1.06666684e-02, -1.48148239e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[-3.61445546e-02, -2.33918279e-02,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-3.61445844e-02, -1.75438598e-02,  0.00000000e+00]],

       [[-1.53273121e-02, -4.92644310e-03,  0.00000000e+00],
        [-2.42650807e-02, -2.16735601e-02,  0.00000000e+00],
        [-1.40480623e-02, -1.67495012e-02,  0.00000000e+00],
        ...,
        [-1.66030079e-02, -1.97011232e-03,

In [31]:
X[:,99,2] = 1

In [32]:
X

array([[[ 2.66666710e-02, -9.87653434e-03,  0.00000000e+00],
        [ 1.86666548e-02,  0.00000000e+00,  0.00000000e+00],
        [ 2.93333530e-02, -2.46913731e-03,  0.00000000e+00],
        ...,
        [-1.33333206e-02, -1.48148239e-02,  0.00000000e+00],
        [-1.06666684e-02, -1.48148239e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[-3.61445546e-02, -2.33918279e-02,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-3.61445844e-02, -1.75438598e-02,  1.00000000e+00]],

       [[-1.53273121e-02, -4.92644310e-03,  0.00000000e+00],
        [-2.42650807e-02, -2.16735601e-02,  0.00000000e+00],
        [-1.40480623e-02, -1.67495012e-02,  0.00000000e+00],
        ...,
        [-1.66030079e-02, -1.97011232e-03,

In [33]:
y.shape

(4000, 4)

In [34]:
#setting up model
model = keras.models.Sequential()
model.add(keras.layers.Convolution1D(128,(6),activation='relu', input_shape=(100,3)))
model.add(keras.layers.Convolution1D(64,(3),activation='relu'))
model.add(keras.layers.Convolution1D(1, (1), activation='relu'))
model.add(keras.layers.LSTM(128, return_sequences=True))
model.add(keras.layers.LSTM(128, return_sequences=True))
model.add(keras.layers.LSTM(128, return_sequences=False))
model.add(keras.layers.Dropout(0.4))
model.add(Dense(y.shape[1]))
# model.add(Dense(13))
model.add(keras.layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_6 (Conv1D)           (None, 95, 128)           2432      
                                                                 
 conv1d_7 (Conv1D)           (None, 93, 64)            24640     
                                                                 
 conv1d_8 (Conv1D)           (None, 93, 1)             65        
                                                                 
 lstm_6 (LSTM)               (None, 93, 128)           66560     
                                                                 
 lstm_7 (LSTM)               (None, 93, 128)           131584    
                                                                 
 lstm_8 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 128)              

In [38]:
X

array([[[ 2.66666710e-02, -9.87653434e-03,  0.00000000e+00],
        [ 1.86666548e-02,  0.00000000e+00,  0.00000000e+00],
        [ 2.93333530e-02, -2.46913731e-03,  0.00000000e+00],
        ...,
        [-1.33333206e-02, -1.48148239e-02,  0.00000000e+00],
        [-1.06666684e-02, -1.48148239e-02,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]],

       [[-3.61445546e-02, -2.33918279e-02,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        [-3.01204920e-02,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-6.02409244e-03, -2.92397663e-02,  0.00000000e+00],
        [-3.61445844e-02, -1.75438598e-02,  1.00000000e+00]],

       [[-1.53273121e-02, -4.92644310e-03,  0.00000000e+00],
        [-2.42650807e-02, -2.16735601e-02,  0.00000000e+00],
        [-1.40480623e-02, -1.67495012e-02,  0.00000000e+00],
        ...,
        [-1.66030079e-02, -1.97011232e-03,

In [36]:
y.shape

(4000, 4)

In [40]:
checkpoint = [keras.callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/models.hdf5')]

In [41]:
mod = model.fit(X, y, epochs=100, batch_size= 100, callbacks= checkpoint, shuffle = True, validation_split=0.1)

Epoch 1/100
36/36 [==============================] - 2s 53ms/step - loss: 1.3670 - accuracy: 0.2800 - val_loss: 1.7901 - val_accuracy: 0.0000e+00
Epoch 2/100
36/36 [==============================] - 1s 37ms/step - loss: 1.3672 - accuracy: 0.2794 - val_loss: 1.8289 - val_accuracy: 0.0000e+00
Epoch 3/100
36/36 [==============================] - 1s 39ms/step - loss: 1.3667 - accuracy: 0.2783 - val_loss: 1.7699 - val_accuracy: 0.0000e+00
Epoch 4/100
35/36 [============================>.] - ETA: 0s - loss: 1.3686 - accuracy: 0.2726

KeyboardInterrupt: 

In [ ]:
total_data= np.array(total_data)

In [ ]:
pd.DataFrame(total_data)

ValueError: Must pass 2-d input. shape=(345, 1000, 655, 3)

In [ ]:
max_length = result_df.Strokes.apply(len).max()

padded_strokes = result_df.Strokes.apply(lambda x: x + [(0, 0, 0)] * (max_length - len(x)))

NameError: name 'result_df' is not defined

In [ ]:
#model construction
model = keras.Sequential()
model.add(layers.LSTM(64, input_shape=(None, 28)))
model.add(Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                23808     
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 10)                650       
                                                                 
Total params: 24714 (96.54 KB)
Trainable params: 24586 (96.04 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________
None
